    1. pre process document text
    2. embed processed tokens
    3. input into dnn trained on words2vec
    4. output tags
    5. store with other lessons in sparse matrix

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
import pickle
import numpy as np
import pandas as pd

In [8]:
latent_topics_list = pickle.load(open('latent_topics_list.pkl', 'rb'))

In [9]:
latent_topics_list

[[(7, 0.22311877), (11, 0.016061254), (12, 0.7606168)],
 [(0, 0.3279571), (7, 0.26539016), (11, 0.011616293), (12, 0.39467552)],
 [(7, 0.8360225), (12, 0.1543574)],
 [(7, 0.85427594), (8, 0.012568254), (12, 0.12065498)],
 [(7, 0.9938185)],
 [(0, 0.056880902),
  (5, 0.06048814),
  (7, 0.3138831),
  (8, 0.082358845),
  (10, 0.07422062),
  (13, 0.40579224)],
 [(7, 0.20555012), (12, 0.16070084), (13, 0.6065019)],
 [(7, 0.5173623), (11, 0.021522876), (12, 0.37942895), (13, 0.07117558)],
 [(7, 0.35375786), (11, 0.026273236), (12, 0.14700477), (13, 0.47276106)],
 [(7, 0.58035266),
  (8, 0.025124116),
  (11, 0.040127374),
  (12, 0.032314498),
  (13, 0.31881517)],
 [(5, 0.035148088),
  (7, 0.122522846),
  (11, 0.062303126),
  (12, 0.5410645),
  (13, 0.23877428)],
 [(0, 0.040206052),
  (5, 0.08738347),
  (6, 0.038410254),
  (7, 0.15267983),
  (10, 0.023296343),
  (13, 0.64981484)],
 [(0, 0.066666596),
  (1, 0.066666886),
  (2, 0.06666661),
  (3, 0.0666667),
  (4, 0.066666596),
  (5, 0.06666659),

In [12]:
def make_sparse_topics(latent_topics_list):
    """
    input:
    latent_topics_list: a list of lists of tuples, what's returned when calling list(lda_model.get_document_topics(bow_corpus)), a list of tuples where the first number
    is the document index, the second number is the document, and for our short term purposes, the index of the of the nested list
    is the document id
    
    outputs:
    3 np arrays to assemble a sparse matrix that has the lesson id, topic number, and the associated value
    
    lesson_ids: list of type int
    topic_ids: list of type int (value between 0 and 14)
    values: list of type float
    
    assumes every lesson has at least one topic value
    
    """
    lesson_ids = []
    topic_ids = []
    values = []
    for idx, lesson in enumerate(latent_topics_list):        
        
        for topic, val in lesson:
            lesson_ids.append(idx)
            topic_ids.append(topic)
            values.append(val)
            
    return np.array(lesson_ids), np.array(topic_ids), np.array(values)
        
row_ind, col_ind, data = make_sparse_topics(latent_topics_list)
M = row_ind.shape[0]
N = col_ind.shape[0]

sparse_arr = np.array([row_ind, col_ind, data])
sid = pd.DataFrame(data=sparse_arr.T, columns=['row_idx', 'col_idx', 'values'])

In [18]:
sid.rename(columns={'row_idx': 'lesson_id', 'col_idx':'topic_id'})

,lesson_id,topic_id,values
0,0.0,7.0,0.223119
1,0.0,11.0,0.016061
2,0.0,12.0,0.760617
3,1.0,0.0,0.327957
4,1.0,7.0,0.265390
...,...,...,...
2991,499.0,0.0,0.184150
2992,499.0,5.0,0.015290
2993,499.0,7.0,0.621754
2994,499.0,8.0,0.038750


In [13]:
def build_sparse_tensor(sparse_idx_df, M,N):
    """
    builds a tf sparse matrix of dim number of M by N, eg. Users by Items, when considering ratings, 
    or lessons by topics when considering topics
    
    Args:
    reviews_df: a pd.DataFrame with `row_idx`, `col_idx` and `values` columns.
    Returns:
    A tf.SparseTensor representing the reviews matrix.
    """

    indices = sparse_idx_df[['row_idx', 'col_idx']].values
    values = sparse_idx_df['values'].values
    
    return tf.SparseTensor(
      indices=indices,
      values=values,
      dense_shape=[M, N])

STFM = build_sparse_tensor(sid, M, N)

In [14]:
STFM

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np

In [3]:
#deep learning library
from keras.models import *
from keras.layers import *
from keras.callbacks import *